In [27]:
import datetime
import pandas as pd
import numpy as np
import math, sys, os
import scipy.stats

filename = 'data/dataset-labeled.csv'
dataset = pd.read_csv(filename)
print('data loaded')

data loaded


In [ ]:
up = dataset[dataset.change>0].shape[0]
down = dataset[dataset.change<0].shape[0]
total = dataset.shape[0]
print(up/down)
print(up/total)
print(down/total)

In [ ]:
upc = dataset[dataset.change>0]['change'].sum()
downc = -dataset[dataset.change<0]['change'].sum()
print(upc, downc, upc/downc)

In [ ]:
stat = pd.DataFrame()
stat['up']=dataset[['security','change']][dataset.change>0].groupby(['security']).count()['change'].astype('i')
stat['down']=dataset[['security','change']][dataset.change<0].groupby(['security']).count()['change'].astype('i')
stat['rate'] = stat['up']/stat['down']
stat.dropna(inplace=True)

scipy.stats.describe(stat['rate'])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

stat = stat.sort_values(by=['rate'])
plt.rcParams['figure.figsize'] = [20, 5]
ax = stat['rate'].plot.bar()
ax.set_xticklabels([])
ax.set_ylim(0,1.5)

ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.2))
ax.set_xticks([])

ax.grid(color='gray',which='major',linestyle='dashed',alpha=0.9)
ax.grid(color='gray',which='minor',linestyle='dashed',alpha=0.65)
plt.show()

In [ ]:
stat[stat.rate>1.15].shape[0]/stat.shape[0]

In [ ]:
dataset[dataset.action=='buy'].shape[0]/ total

In [2]:
from lib.jqdata import *
from lib.func import *
np.random.seed(2)
security_list = get_all_securites().sample(1)
security = security_list['security'].iloc[0]

history = dataset[dataset.security==security].copy()
print(security, history.shape)

def stat_gn(v):
    return v[v<0].shape[0]/(len(v)-(v[v==0].shape[0]))

pn = get_prime_numbers(2,150)

for w in pn:
    history['r_{}'.format(w)] = history['change'].rolling(window=w).apply(func=stat_gn, raw=False)
    print(w)
    
history.to_csv('data/stat-{}.csv'.format(security))
print('done')

600648.XSHG (967, 12)
2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101
103
107
109
113
127
131
137
139
149
done


In [31]:
history=history.dropna()
for w in pn:
    factor = history['r_{}'.format(w)]
    st = scipy.stats.describe(factor)
    print("period:{:03d}\trange:{:.3f} {:.3f}-{:.3f} {:.3f} \tmean:{:.3f}\t{:.4f}\t{:.3f}\t{:.3f}".format(w, 
      st.minmax[0],factor.quantile(0.25),factor.quantile(0.75),st.minmax[1],st.mean,st.variance, st.kurtosis, st.skewness))

period:002	range:0.000 0.000-1.000 1.000 	mean:0.506	0.1891	-1.660	-0.024
period:003	range:0.000 0.333-0.667 1.000 	mean:0.477	0.1237	-1.165	0.119
period:005	range:0.000 0.200-0.600 1.000 	mean:0.464	0.0758	-0.682	0.375
period:007	range:0.000 0.286-0.571 1.000 	mean:0.455	0.0464	-0.297	0.146
period:011	range:0.000 0.364-0.545 0.818 	mean:0.455	0.0248	0.138	-0.194
period:013	range:0.000 0.385-0.538 0.769 	mean:0.450	0.0188	0.955	-0.453
period:017	range:0.000 0.412-0.529 0.765 	mean:0.461	0.0163	1.133	-0.576
period:019	range:0.105 0.421-0.526 0.684 	mean:0.462	0.0112	0.783	-0.602
period:023	range:0.174 0.400-0.522 0.652 	mean:0.471	0.0081	0.734	-0.696
period:029	range:0.276 0.414-0.517 0.690 	mean:0.473	0.0061	0.233	-0.147
period:031	range:0.258 0.419-0.516 0.677 	mean:0.472	0.0051	0.615	-0.039
period:037	range:0.270 0.432-0.514 0.649 	mean:0.469	0.0050	0.435	-0.169
period:041	range:0.317 0.439-0.506 0.634 	mean:0.469	0.0041	0.292	0.010
period:043	range:0.326 0.442-0.512 0.628 	mean:0.46

In [86]:
from lib.jqdata import *
from lib.func import *
np.random.seed(2)
security_list = get_all_securites().sample(1)
security = security_list['security'].iloc[0]

history = dataset[dataset.security==security].copy()
print(security, history.shape)

history['fu_2'] = history['change'].shift(periods=-2)
history['fu_1'] = history['change'].shift(periods=-1)
history['pre_1'] = history['change'].shift(periods=1)
history['pre_2'] = history['change'].shift(periods=2)
history['pre_3'] = history['change'].shift(periods=3)
history['pre_4'] = history['change'].shift(periods=4)
history['pre_5'] = history['change'].shift(periods=5)
history['pre_6'] = history['change'].shift(periods=5)

# query = "pre_5>0 & pre_4>0 & pre_3>0 & pre_2<0 & pre_1<0 & change>0"
query = "pre_1<0 & change>0" #0.54
query = "change<0" #0.55
# query = "pre_4<0 & pre_3<0 & pre_2<0 & pre_1<0 & change<0" #0.60
# query = "pre_5<0 & pre_4<0 & pre_3<0 & pre_2<0 & pre_1<0 & change<0" #0.66
# query = "pre_5<0 & pre_4<0 & pre_3<0 & pre_2<0 & pre_1<0 & change>0" #0.69
# query = "pre_4<0 & pre_3<0 & pre_2<0 & pre_1<0 & change>0" #0.56

subset = history[history.eval(query)]
print(history[['pre_1','change','fu_2']])
print( len(subset))
print( len(subset[subset.fu_1>=0])/len(subset))

600648.XSHG (967, 12)
           pre_1    change      fu_2
756937       NaN       NaN  0.006033
756938       NaN -0.023564  0.001499
756939 -0.023564  0.006033 -0.038922
756940  0.006033  0.001499 -0.024922
756941  0.001499 -0.038922 -0.014377
...          ...       ...       ...
757899 -0.010411  0.005260 -0.005656
757900  0.005260  0.017792 -0.002585
757901  0.017792 -0.005656 -0.009331
757902 -0.005656 -0.002585       NaN
757903 -0.002585 -0.009331       NaN

[967 rows x 3 columns]
452
0.5508849557522124
